# Automated metadata matching

Life sciences (LS)/ Clinical research institutes (academic research institutes, pharma companies, hospitals, clinics etc.) across the world are producing large volumes of data from patients. This can range from clinical information such as diagnostic/prognostic data, omic data such as genetic/proteomic/epigenetic screens, pathological data such as MRI scans etc. One of the main objectives in LS research (both academic and industrial) is to gain actionable insights from these data sets, that goes beyond the diagnosis/prognosis of a (group of) patient(s) and provides a deeper understanding of the diseases, as well as shine lights on new therapeutic options. It is becoming apparent, that to gain actionable insights from LS data sets, we need data from a large number of patients. This is achievable, if we could merge datasets from various institutes, which is turning out to be hugely challenging task, simply because different institutes use different standards, units, nomenclature etc. to store data. <br><br>

For instance, patient's age is a common clinical parameter recorded by almost all organisations. One institute can name the variable that records patients’ age as 'patient age', another can name the same variable as 'age', others can name it as 'age at diagnosis', 'days since birth', 'years since birth' etc. The values can also be in days, months, years etc. Therefore, to combine data from many institutes (and sometimes within same institutes), it's essential to understand that all of the above variables are recording the same thing, i.e. patient's age, also we need to make sure that the units (days, months, years) of measuring age are homogenised at the time of integration. <br><br>

To assist in the above process the National Cancer Institute (NCI) created the concept of CDE (common data element). See https://cdebrowser.nci.nih.gov/cdebrowserClient/cdeBrowser.html#/search for more details. A big data dump of about 69000 CDE elements are provided in 'cde_database\full_database' folder in XML format, if you want to further explore. They provide a standard format of representing Life Science's data. This gives us standard variable name, the permissible values , units etc. for each of these clinical parameters. Some research organizations are following this standard, but vast majority aren't. Additionally, there is huge amount of data produced until now which are not standardized using CDEs. <br><br>

To be able to integrate data from various institutes, we need to be able to match the variable names in the clinical datasets to the corresponding CDE elements. Currently, there is a drive for developing ML/AI algorithms to achieve this.<br><br>


The code below is an initial attempt in this direction. In summary, it tries to match the variables names (generally the column headers in a clinical data file) and values (the column values) of the clinical parameters in a dataset, to the long variable names, and permissible values of the CDE elements. The objective is to find the CDE elements that closely match the each clinical parameter name (i.e. the column header). To do the the following steps are performed: <br><br>

1. Converted selected aspects (e.g. long_name, permissible_values etc.) of all CDE elements into numerical vectors using a word embedding model which itself was trained on these data.

2. Coverted the clinical parameter names (headers) and values into numerical vector using the same word embedding model as above.

3. The vectors from the clinical data can be matched to CDE vectors in a few different ways: <br>
   (a) One way is to use unsupervised learning, fit a Nearest Neighbour model to the CDE vectors, and look for the nearest neighbors of each clinical parameter using this model.
   (b) Another way is to use supervised learning: Create feature vectors for all possible pairs of clinical parameters and CDE elements, consider the true pairs as positive class (target =1) and the remaining pairs as negative class (target = 0). Train classifiers to this data and use the classifier to evaluate new clinical parameters.
   
 
See more details below.
   
   
   
   





## Install custom benchmark solutions libraries

In [1]:
!pip install cde_modelling_tools/.

Processing c:\users\mark.kirby\accenture\life sciences hackathon - hackathon channel\hackathon data\cde_modelling_tools
  Created wheel for cde-data-modeller: filename=cde_data_modeller-0.0.1-py3-none-any.whl size=15911 sha256=009054b43a1c2cc8da641258bf05178ea1f052b2a1b5ad9ba6f8159f341b6360
  Stored in directory: C:\Users\MARK~1.KIR\AppData\Local\Temp\pip-ephem-wheel-cache-4frg3nlz\wheels\bb\b2\30\93890ab049c5c3380348a91a0b0cf591e584e44a0c5a0c727b
Successfully built cde-data-modeller
  Attempting uninstall: cde-data-modeller
    Found existing installation: cde-data-modeller 0.0.1
    Uninstalling cde-data-modeller-0.0.1:
      Successfully uninstalled cde-data-modeller-0.0.1


In [2]:
!pip install -r requirements.txt

## Import necessary libraries

In [3]:
import pandas as pd

import json
import numpy as np
import random
import mlflow
from cde_modelling.modelling import CDE_data_modeller as cdm
from cde_modelling.parsing import TCGA_data_parser as tdp
from cde_modelling.utils import Accuracy_calculations as ac
import pickle 
from sklearn.model_selection import train_test_split

C:\Users\mark.kirby\Anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


## File paths

In [4]:
clinical_data_files_dir = 'tcga_training_data/'

clinical_data_test_dir = 'tcga_test_data/'

cde_database_file = 'cde_database/combined_small_dataset.json'

parameter_file = 'params_supervised.json'

model_dir = 'models/'

test_gold_standard = 'gold_standard/test_gs.json'

## Load model parameters

In [5]:
# read model parameters
params = {}

with open(parameter_file,'r') as file:
    params = json.load(file)


## Create a fasttext model for the CDE database and index the individual CDE elements in the database 

Fasttext is a word embedding algorithm developed by FaceBook. Given a corpus, it creates a model that tries to predict if a pair of words appear in the same context. The model first converts the words to a numeric vector which are used as features for the above prediction. We are interested in the feature generation part, i.e. the part which converts words to numeric vectors. For more information on the FastText model see https://radimrehurek.com/gensim/models/fasttext.html.

### FastText model training: 

To train a FastText model we first extracted the long_name and permissible_values of each CDE elements. These were then parseed and cleaned (lower cased, alphanumeric character only, splitted into bag of words). The preprocessed long names and permissible values of all CDE element was considered as the training corpus for the FastText model. The corpus was then used to train A FastText model. The parameters for the model are in the above json file. The trained model is then used to index the CDE elements (i.e. create numeric vectors representing each CDE). We created two sets of vectors for CDE elements, one for the long_names and the other for permissible values. We alo extracted the data_type information for each CDE elements. Below is an example. Let's assume that the following is a (oversimplified) CDE element .
CDE_element: 
{
'public_id': 1234
'long_name': 'received radiotherapy'
..............
'permissible_values': ['yes','no']
}.

To index the above CDE, we performed the following:

1. Vectorized the long_name entry (i.e. 'received radiotherapy') using the FastText model. To do that, we vectorized each word (i.e. 'received' and 'radiotherapy') of the long name entry separately. The vectors were then normalized by their L2 norms and averaged. Say for example, the long_name vector is [0.1, 0.345]. 

2. Vectorized the 'permissible_values' entry (i.e. 'yes', 'no') using the FastText model. To do that, we vectorized each word (i.e. 'received' and 'radiotherapy') in the permissible_values entry separately. The vectors were then normalized by their L2 norms and averaged.Say for example, the permissible vector is [0.981, 0.233]. 

3. We identified whether the permissible values are string or numbers. Note, that for the benchmark solution, we kept this simple. But for the hackathon, the participants can conder more grannular data type for example, string, binary, float, int long etc.


Combination of the above is used to numerically represent (index) each CDE. The class CDE_data_modeller, in package cde_modelling_tools does the above. Pparticipants should explore using other entries in the CDE data fields to improve their chances of finding a match.

The CDE_data_modeller class not only creates the word embedding models and index (vectorize) the CDE data elements, it can also save and load pretrained models and indexes.

In [10]:
cde_data_modellers = cdm.CDE_data_modeller(cde_database_file, params)
cde_data_modellers.create_model_and_cde_indexes()
# cde_data_modellers.save_model_and_indexes(model_dir+'fasttext/')

Loading CDE database... please wait
Took 0.001596  minutes to load CDE database..
Starting model training ... 


 20%|███████████████▎                                                           | 1019/4991 [00:00<00:00, 10111.28it/s]

Model training took 8.143119 minutes
Start converting descriptors to vectors


  0%|                                                                                         | 0/4991 [00:00<?, ?it/s]C:\Users\mark.kirby\Anaconda3\lib\site-packages\cde_modelling\modelling\CDE_data_modeller.py:498: RuntimeWarning: invalid value encountered in true_divide
  vector = vector/len(sentence)


Took 0.009076 minutes to vectorize the dataset
Start converting descriptors to vectors


100%|████████████████████████████████████████████████████████████████████████████| 4991/4991 [00:00<00:00, 5329.46it/s]

Took 0.015975 minutes to vectorize the dataset


## Load a pretrained FastText model and saved indexes for CDE elements

In [25]:
cde_data_modellers = cdm.CDE_data_modeller(cde_database_file, params)
cde_data_modellers.load_model_and_cde_indexes(model_dir+'fasttext/')

Loading CDE database... please wait
Took 0.001695  minutes to load CDE database..


## Load and parse training data

The training data are a set of clinical data files which records cinical information of patients, e.g. gender, age, disease_type, disease_sub_type, treatment received etc. It's in table format, where the rows represent patients and the columns represent colinical parameters. In case of the training data, the CDE data element corrsponding to each clinical parameter is provided. This information can be used to train machine learning algorithms to predict CDE elements for new clinical parameters.

In [26]:
tdpr = tdp.TCGA_data_processor(clinical_data_files_dir,True )
tcga_data = tdpr.get_parsed_data()


  2%|█▏                                                                                | 3/200 [00:00<00:07, 25.71it/s]

 Processing clinical metadata.. please wait..


100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:04<00:00, 42.71it/s]


The parser returns three types of information for each clinical parameter.

1. The name of the parameter (e.g. age, gender, etc.)
2. List of values for each parameters (except id columns, continuous variabales etc.)
3. Data type of the values. For instance, data type of 'age' is 'number', data type of gender = 'string'. 
4. A dictionary containing clinical parameters and it's corresponding 

See the parsed data below.

In [27]:
tcga_data.keys()

dict_keys(['headers', 'values', 'value_type', 'gold_standard'])

In [28]:
tcga_data['headers'] # tcga_data['values'] / tcga_data['value_type'] / tcga_data['gold_standard']

{'bcr_patient_barcode': ['bcr', 'patient', 'barcode'],
 'bcr_drug_barcode': ['bcr', 'drug', 'barcode'],
 'bcr_drug_uuid': ['bcr', 'drug', 'uuid'],
 'form_completion_date': ['form', 'completion', 'date'],
 'pharmaceutical_therapy_drug_name': ['pharmaceutical',
  'therapy',
  'drug',
  'name'],
 'clinical_trial_drug_classification': ['clinical',
  'trial',
  'drug',
  'classification'],
 'pharmaceutical_therapy_type': ['pharmaceutical', 'therapy', 'type'],
 'pharmaceutical_tx_started_days_to': ['pharmaceutical',
  'tx',
  'started',
  'days',
  'to'],
 'pharmaceutical_tx_ongoing_indicator': ['pharmaceutical',
  'tx',
  'ongoing',
  'indicator'],
 'pharmaceutical_tx_ended_days_to': ['pharmaceutical',
  'tx',
  'ended',
  'days',
  'to'],
 'treatment_best_response': ['treatment', 'best', 'response'],
 'days_to_stem_cell_transplantation': ['days',
  'to',
  'stem',
  'cell',
  'transplantation'],
 'number_cycles': ['number', 'cycles'],
 'pharm_regimen': ['pharm', 'regimen'],
 'pharm_regimen

## Create base tables for model training

To create base tables I performed the following:

1. Indexed (vectorized using the FastText model) the headers (clinical parameter names) and values of each clinical parameters parsed in the previous step.
2. for each possible pair of clinical_parameter and CDE elements we calculate the following features <br>
    (a) Difference between the embedding vectors of the CDE long_name and the clinical parameter name. <br>
    (b) Difference between the embedding vectors of the CDE permissible values and the values associated with the clinical parameters in the training dataset. <br>
    (c) A similarity measure (cosine similarity, correlation etc.) betwween the CDE long_name and clinical parameter name vectors <br>
    (d) A similarity measure (cosine similarity, correlation etc.) betwween the CDE permissible_vaue and observed clinical parameter value vectors <br>
    (e) Similaritied between the data type of the permissible and observed values of the CDE and the observed clinical oaraneters respectively <br>

3. Note that, in the base table one data point is represented by a pair (clinical parameter and a CDE ). For example: If there are 800 cinical parameters in the training data and 5000 CDE elements in the CDE dataset, the the base table will have 500*8000 = 4million entries. Each entry will have the above features. The 'target' variable is defined as follows: <br>

$
target = 1, \text{if the CDE element is manually matched to the clinical parameter} \\
target = 0, \text{otherwise}
$

In the above example, there are 800 clinical parameters, and if only 1 CDE elements is matched to each clinical parameter, the target variable can be equals to 1 in only 800 out of 4 million cases. Therefore the base table is extremely imbalanced. To counter this we need to undersample (or oversample) the abt. The create_abt function in CDE_data_modeller allows undersampling. The ratio of undersampling (number of cases target = 0 / number of cases target =1 ) can be adjusted using the params dictionary. The defalut value is 5 which means in the undersampled base tables, 16.67 % of cases have target =1 and 83.33% of cases have target = 0.





In [29]:
abt = cde_data_modellers.create_abt(tcga_data, params)

  0%|                                                                                          | 0/701 [00:00<?, ?it/s]

Start converting descriptors to vectors
Took 0.001197 minutes to vectorize the dataset
Start converting descriptors to vectors


100%|███████████████████████████████████████████████████████████████████████████████| 701/701 [00:01<00:00, 567.75it/s]


Took 0.020711 minutes to vectorize the dataset


In [30]:
abt.head()

,feature_vec_0_x,feature_vec_1_x,feature_vec_2_x,feature_vec_3_x,feature_vec_4_x,feature_vec_5_x,feature_vec_6_x,feature_vec_7_x,feature_vec_8_x,feature_vec_9_x,...,feature_vec_44_y,feature_vec_45_y,feature_vec_46_y,feature_vec_47_y,feature_vec_48_y,feature_vec_49_y,header_metrics,value_metrics,metric,target
20019,-0.074879,0.044559,-0.151783,-0.005449,-0.010106,-0.151557,-0.010543,-0.081013,-0.057839,0.023783,...,-0.020757,-0.052062,0.051249,0.013284,-0.074078,-0.016504,0.750432,0.728281,0.0,1.0
25204,0.003034,0.001879,0.029784,0.058840,0.013281,0.039684,0.030796,0.025707,0.029569,-0.026366,...,-0.037163,-0.024893,0.044011,-0.020780,-0.072418,-0.000053,0.898956,0.785171,0.0,1.0
30658,-0.047950,0.021755,0.013564,-0.008411,0.107960,-0.055107,-0.035221,-0.002115,-0.013555,0.088978,...,-0.016799,0.021785,0.007958,0.035195,0.035951,0.008849,0.784019,0.745251,0.0,1.0
35381,-0.017392,-0.023105,-0.014109,-0.058086,-0.075214,-0.057065,0.057395,-0.120960,0.008653,-0.081426,...,-0.035563,-0.058119,0.100232,-0.017497,0.045034,-0.001179,0.793131,0.716037,0.0,1.0
35382,-0.017392,-0.023105,-0.014109,-0.058086,-0.075214,-0.057065,0.057395,-0.120960,0.008653,-0.081426,...,-0.035563,-0.058119,0.100232,-0.017497,0.045034,-0.001179,0.793131,0.716037,1.0,1.0


## Train a machine learning model using the abt created above
I created a separate class called create_model where a number of supervised and unsupervised learning algorithms are implemented (from sklearn library). The type and parameters of the model can be passed using the params dictionary. 

!!! Warning: Currently the the two datatype columns in the abt (data_type_string, data_type_number) are complementary and hence redundant. Only one should be used for modelling. This needs to be corrected in the future versions.

In [31]:
# First create feature vectors
features = [c for c in abt.columns if ('feature' in c) or ('metric' in c)]

# create design matrix and target data
X = abt[features]
y= abt['target']

# create training and validation data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state=42)


# import the Model class
from cde_modelling.modelling.create_models import Model

# create model
model = Model(params)

#fit the model
model.fit(X_train,y_train)

# calculate the accuracy of the model on the validation data
accuracy = model.accuracy(X_val, y_val)

In [32]:
accuracy

{'accuracy': 0.9299287410926366,
 'f1': 0.8052805280528054,
 'precision': 0.8652482269503546,
 'recall': 0.7530864197530864,
 'auroc': 0.9692992011619463,
 'confusion_matrix': array([[661,  19],
        [ 40, 122]], dtype=int64),
 'matthews': 0.7655409042452315}

## Test the model on the test dataset

To do that, we shall first create the base table for the test dataset by parsing and indexing the test set in the same way as was done for the training set.

In [33]:
tdp1 = tdp.TCGA_data_processor(clinical_data_test_dir,False )
test_data = tdp1.get_parsed_data()
test_abt =cde_data_modellers.create_abt(test_data)

  0%|                                                                                            | 0/6 [00:00<?, ?it/s]

 Processing clinical metadata.. please wait..


 18%|██████████████▏                                                                 | 40/225 [00:00<00:00, 358.10it/s]

Start converting descriptors to vectors


  0%|                                                                                          | 0/225 [00:00<?, ?it/s]

Took 0.005917 minutes to vectorize the dataset
Start converting descriptors to vectors


100%|███████████████████████████████████████████████████████████████████████████████| 225/225 [00:01<00:00, 131.16it/s]


Took 0.028640 minutes to vectorize the dataset


## Make predictions for the test dataset using the trained model

Note that I have created a model.predict_and_convert_to_json function which returns the prediction in the following format: <br>
{
clinical parameter1: [most likely predictions, 2nd most likely prediction, .... , 20th most likely prediction] <br>
clinical parameter2: [most likely predictions, 2nd most likely prediction, .... , 20th most likely prediction] <br>
.....
clinical parametern: [most likely predictions, 2nd most likely prediction, .... , 20th most likely prediction] <br>
}

In [34]:
test_abt.fillna(0, inplace = True)

index_cols = ['headers','public_id']
header_col = index_cols[0]
id_col = index_cols [1]

results = model.predict_and_convert_to_json(test_abt,20, index_cols, header_col, id_col)


In [35]:
results

{'abnormal_lymphocyte_percent': ['3144381',
  '3206020',
  '2841253',
  '6760770',
  '3151235',
  '4851631',
  '3293767',
  '5543316',
  '3094883',
  '5010431',
  '2669789',
  '3366911',
  '3613492',
  '3174022',
  '3151753',
  '3151599',
  '3203106',
  '5178437',
  '3203222',
  '3733535'],
 'age_at_first_diagnosis': ['3165480',
  '3457768',
  '3414609',
  '3414617',
  '4618491',
  '3462301',
  '3414621',
  '3225640',
  '3414503',
  '3008273',
  '3131740',
  '2282',
  '6943594',
  '3008335',
  '3440606',
  '3430945',
  '4887491',
  '3457771',
  '3414554',
  '6982687'],
 'age_at_initial_diagnosis': ['3523210',
  '3523205',
  '4618491',
  '3462301',
  '3131740',
  '3414617',
  '3430945',
  '3008335',
  '6943594',
  '3479270',
  '3284270',
  '3165480',
  '3440606',
  '3408682',
  '3008313',
  '3527997',
  '3414609',
  '3151116',
  '3225640',
  '3008273'],
 'age_upon_diagnosis': ['3284270',
  '3284879',
  '2006657',
  '3225640',
  '2939739',
  '6982687',
  '3629991',
  '3966995',
  '362999

# Calculate accuracy of prediction for the test dataset

Note that the participants won't have access to the gold standard data, therefore won't be able to perform the following step. However, participants can divide the training data in to train, test, validation sets and perform the following on the test data.

In [12]:
test_gs = {}
with open(test_gold_standard, 'rb') as file:
    test_gs = json.load(file)
test_accuracy = ac.calculate_accuracy(test_gs,results)

In [13]:
test_accuracy

0.46000000000000013

## Log model parameters etc. using mlflow

This will ensure reproducibility of results and will keep track of all models and results during the model development and calibration.

In [14]:
with mlflow.start_run():
    # print out current run_uuid
    run_uuid = mlflow.active_run().info.run_uuid
    print("MLflow Run ID: %s" % run_uuid)
    
    # log parameters
    mlflow.log_param("window_size", params["fasttext"]["window"])
    mlflow.log_param("min_count", params["fasttext"]["min_count"])
    mlflow.log_param("epochs", params["fasttext"]["epochs"])
    mlflow.log_param("vector_size", params["fasttext"]["vector_size"])
    
    
    mlflow.log_param("features_diference_types", params["features"]["differences"]["type"])
    mlflow.log_param("features_metrics", params["features"]["metrics"]["metric"])
    mlflow.log_param("features_metrics_sim_type", params["features"]["metrics"]["sim_type"])
    mlflow.log_param("features_metrics_scaling", params["features"]["metrics"]["scaling"])
    mlflow.log_param("features_sampling_ratio", params["features"]["sampling_ratio"])
    
    mlflow.log_param("features_samplinf_ratio", params["features"]["sampling_ratio"])
    
    mlflow.log_param("model_type", params['model']["name"])
    
    for k in params['model']['model_params'].keys():
        mlflow.log_param("model_params_"+k, params['model']["model_params"][k])
    
    # log metrics
        
    mlflow.log_metric("test_accuracy",test_accuracy)
    for k in accuracy.keys():
        if 'confusion' not in k:
            mlflow.log_metric("val_accuracy_"+k,accuracy[k])
    
    #mlflow.sklearn.logmodel()
    with open('models/'+run_uuid+'.pkl','wb') as file:
        pickle.dump(model, file)
    
    mlflow.end_run()

MLflow Run ID: be94df1a6dfe4b1da9d06d797bfca041


## Use 'mlflow ui' to compare and analyze various model performances

<img src="img/mlflow_ui.png">

In [ ]:
!mlflow ui

### To view mlflow ui go to http://localhost:5000